In [1]:
import os
import numpy as np
from spectral.io import envi
import pandas as pd

%matplotlib inline

In [2]:
df_noise_vectors = pd.DataFrame()

In [3]:
srcFolder = '/Volume1/data/CRISM/yuki/v5/'
for r, d, f in os.walk(srcFolder):
    for file in f:
        if file.find('_nr_ds.img') != -1:
            'The image name is'
            imgName = os.path.join(r, file)
            print imgName
            hdrName = imgName.replace('.img', '.hdr')
            img = envi.open(hdrName, imgName)
            img = img.load()
            img = img[:, :, 4:244]
            [rows, cols, bands] = img.shape
            
            'The background image is'
            bgImgName = imgName.replace('_nr_ds', '_Bg_ds')
            bgHdrName = bgImgName.replace('.img', '.hdr')
            bgImg = envi.open(bgHdrName, bgImgName)
            bgImg = bgImg.load()
            bgImg = bgImg[:, :, 4:244]
            
            'The absorption image is'
            abImgName = imgName.replace('_nr_ds', '_AB_ds')
            abHdrName = abImgName.replace('.img', '.hdr')
            abImg = envi.open(abHdrName, abImgName)
            abImg = abImg.load()
            abImg = abImg[:, :, 4:244]
            
            'The residual image is'
            resImg = img - (abImg * bgImg)
            
            'convert it to a 2D array'
            resArr = resImg.reshape((rows*cols, bands))
            'make it a pandas array'
            resArr = pd.DataFrame(resArr)
            resArr = resArr.dropna()
            
            'Append it to larger datafrane'
            df_noise_vectors = df_noise_vectors.append(resArr)

/Volume1/data/CRISM/yuki/v5/FRT000047A3/FRT000047A3_07_IF166L_TRRD_sabcondpub_v1_trrd_cor_cbc11_nIter2_tu2_ltn0_ltnl035_nr_ds.img


/usr/local/lib/python2.7/dist-packages/spectral/io/spyfile.py:252: NaNValueWarning: Image data contains NaN values.
  warnings.warn('Image data contains NaN values.', NaNValueWarning)


/Volume1/data/CRISM/yuki/v5/HRL000040FF/HRL000040FF_07_IF183L_TRRD_sabcondpub_v1_lam2_cbc11_40A2_tu2_nr_ds.img
/Volume1/data/CRISM/yuki/v5/FRT00005C5E/FRT00005C5E_07_IF166L_TRRD_sabcondpub_v1_trrd_cor_cbc11_nIter2_tu2_ltn0_ltnl035_nr_ds.img
/Volume1/data/CRISM/yuki/v5/FRT0001FB74/FRT0001FB74_07_IF166L_TRRD_sabcondpub_v1_trrd_cor_cbc11_nIter2_tu2_ltn0_ltnl035_nr_ds.img
/Volume1/data/CRISM/yuki/v5/FRT00009786/FRT00009786_07_IF165L_TRRD_sabcondpub_v1_trrd_cor_cbc11_nIter2_tu2_ltn0_ltnl035_nr_ds.img
/Volume1/data/CRISM/yuki/v5/FRS00031442/FRS00031442_01_IF168L_TRRD_sabcondpub_v1_trrd_cor_cbc11_nIter2_tu2_ltn0_ltnl035_nr_ds.img


In [4]:
print df_noise_vectors.shape
store = pd.HDFStore('/Volume2/arunFiles/python_HSITools/crismBalancingDatasets/dataProducts/store_residualVectors.h5')
store.put('IF_residuals', df_noise_vectors, format='t', append=False)
store.close()

(1398384, 240)
